In [1]:
import os
os.chdir("..") 

In [2]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline, make_union
from utils.Transformers2 import FillNaDict, DFFeatureUnion, ColumnSelector, DFStandardScaler, DummyTransformer, DFKBinsOrdinalDiscretizer, DFOrdinalEncoder
import numpy as np
from utils.MinMaxScalerOutliers import MinMaxScalerQ
from sklearn.model_selection import GridSearchCV
import joblib

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [4]:
seed = 1111

In [5]:
data_path = "data/"

In [6]:
X_train = pd.read_csv(data_path + 'train.csv', sep='\t', encoding='utf-8', )  
X_train.head()

,installments.quantity,installments.rate,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,accepts_mercadopago,category_id,is_official_store,catalog_product_id,tags,catalog_listing,seller.id,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,prices.prices,address.state_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller.seller_reputation.real_level,root_category_id,root_category_name,discount,category_name,categories_from_root
0,12.0,70.17,MLA896914637,MLA,Magnetron Para Microondas Modelo 2m219g 2m 219 G,1850.0,ARS,1,50,buy_it_now,gold_special,new,1,MLA455062,0,NaN,"['brand_verified', 'good_quality_picture', 'go...",0,169949346,0,0,"['normal', 'user_info_verified', 'credits_prio...",22721.0,906.0,historic,0.01,0.98,0.01,21815.0,platinum,0.0053,0.0011,4.0,60 days,60 days,3587.0,0.0029,11.0,60 days,"[{'id': '2', 'type': 'standard', 'conditions':...",Buenos Aires,0,me2,[],cross_docking,0,NaN,MLA5726,Electrodomésticos y Aires Ac.,NaN,Magnetrones,"['Electrodomésticos y Aires Ac.', 'Cocción', '..."
1,12.0,70.17,MLA689849716,MLA,Motor Y Pedal P/máquinas De Coser Familiar Sin...,4890.0,ARS,1,250,buy_it_now,gold_special,new,1,MLA116430,0,NaN,"['good_quality_thumbnail', 'loyalty_discount_e...",0,93137649,0,0,"['normal', 'user_info_verified', 'credits_prio...",4037.0,175.0,historic,0.00,1.00,0.00,3862.0,platinum,0.0000,0.0221,17.0,60 days,60 days,758.0,0.0000,0.0,60 days,"[{'id': '3', 'type': 'standard', 'conditions':...",Buenos Aires,1,me2,"['fulfillment', 'mandatory_free_shipping']",fulfillment,0,NaN,MLA5726,Electrodomésticos y Aires Ac.,NaN,Otros Accesorios,"['Electrodomésticos y Aires Ac.', 'Pequeños El..."
2,NaN,NaN,MLA909338268,MLA,Honda Civic 1.8 Exs At 140cv - 127218 - C(p),1480000.0,ARS,1,0,classified,gold_premium,used,1,MLA1744,0,MLA8713970,"['good_quality_thumbnail', 'good_quality_pictu...",0,439841655,1,0,"['car_dealer', 'user_info_verified', 'messages...",150.0,72.0,historic,0.02,0.93,0.05,78.0,NaN,0.0000,0.0000,0.0,365 days,365 days,78.0,0.0000,0.0,365 days,"[{'id': '2', 'type': 'standard', 'conditions':...",Capital Federal,0,not_specified,[],not_specified,0,NaN,MLA1743,"Autos, Motos y Otros",NaN,Autos y Camionetas,"['Autos, Motos y Otros', 0 ..."
3,12.0,70.17,MLA750172551,MLA,Radio Yongnuo Rf-603 Ver Ii / Nikon O Canon / ...,6688.0,ARS,1,100,buy_it_now,gold_special,new,1,MLA411998,0,NaN,"['good_quality_picture', 'good_quality_thumbna...",0,141498690,0,0,"['normal', 'user_info_verified', 'credits_prio...",14212.0,786.0,historic,0.00,1.00,0.00,13426.0,platinum,0.0057,0.0102,23.0,60 days,60 days,2325.0,0.0000,0.0,60 days,"[{'id': '1', 'type': 'standard', 'conditions':...",Capital Federal,1,me2,"['fulfillment', 'self_service_out', 'mandatory...",fulfillment,0,NaN,MLA1039,Cámaras y Accesorios,NaN,Disparadores,"['Cámaras y Accesorios', 'Accesorios para Cáma..."
4,NaN,NaN,MLA853999531,MLA,"1, 2 Y 3 Ambientes Con Cocheras Opcionales",110000.0,USD,1,0,classified,gold,new,0,MLA401806,0,NaN,['good_quality_picture'],0,102898425,0,1,"['real_estate_agency', 'user

In [7]:
X_train.shape

(23194, 52)

In [8]:
X_train.discount.describe()

count    2818.000000
mean        0.180847
std         0.119029
min         0.000000
25%         0.080000
50%         0.158469
75%         0.259146
max         0.720000
Name: discount, dtype: float64

In [11]:
X_train.price.describe().round(2)

count       23009.00
mean       126819.38
std       1129333.64
min             1.00
25%           820.00
50%          1899.00
75%          5149.00
max      85518000.00
Name: price, dtype: float64

In [14]:
X_train['installments.rate'].describe().round(2)

count    21510.00
mean        63.82
std         20.14
min          0.00
25%         70.17
50%         70.17
75%         70.17
max         70.17
Name: installments.rate, dtype: float64

In [17]:
X_train['installments'].describe().round(2)

KeyError: 'installments'

In [ ]:
X_train.dtypes

In [ ]:
aux = X_train.copy()

## Target

In [ ]:
X_train.sold_quantity.value_counts().sort_index()

<p style="color:red; font-weight: bold">Vamos a intentar probar unificar las categorias iniciales, considerando este tipo de decisiones, en caso de quen hagan que el modelo de mejor, deben consultarse siempre con el negocio </p>

In [ ]:
X_train["sold_quantity"] = [x if x >= 5 else 1 for x in X_train["sold_quantity"]]

In [ ]:
X_train.sold_quantity.value_counts().sort_index()

# y

In [ ]:
y = X_train.sold_quantity

In [ ]:
X_train = X_train.drop(['sold_quantity'], axis= 1)

## corrMatrix - post pipeline

selected_columns = [
    
]

corrMatrix = df[selected_columns].corr()
corrMatrix

sn.heatmap(corrMatrix, annot=True)
plt.show()

## LabelEncoder

enc = LabelEncoder()
enc.fit(y)
y_transformed = enc.transform(y)

pd.Series(y_transformed).value_counts(normalize=True).sort_index().plot.barh()

## SMOTE strategy

In [ ]:
dic = pd.Series(y).value_counts().sort_index().to_dict()
dic

In [ ]:
threshold = 3000
prop = 1.5

In [ ]:
smote_sampling_strategy = {key: int(value*prop) if value < threshold else value for key, value in dic.items()}
smote_sampling_strategy

## IDs  + columnas a descartar

In [ ]:
id_cols = [
'id',
'site_id',
'title',
'category_id',
'seller.id',
'catalog_product_id',
'root_category_id',
'root_category_name',
'category_name',
]

In [ ]:
columns_discard = [
    'tags',
    'seller.tags', 
    'shipping.tags',
    'categories_from_root', 
    'prices.prices'

]

In [ ]:
ids_df = X_train[id_cols].copy()
X_train = X_train.drop(id_cols + columns_discard, axis= 1)

## Columns

In [ ]:
discard_columns = [
    'tags', 
    'categories_from_root',
    'shipping.tags',
]

In [ ]:
cols_preproc = [x for x in X_train.columns if x not in discard_columns] 
cols_preproc

In [ ]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.to_list()
# numeric_features

In [ ]:
categorical_ord_enc = [
    'available_quantity',
    'installments.quantity',
]

In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns.to_list() 
categorical_features

## NA

Nota: para las categóricas habría que consultar con expertos del dominio en cómo completar

In [ ]:
fillNA = {
    'cat': {},
    'num': {},
}
for x in numeric_features:
    fillNA['num'][x] = 0 #'__NA__'
    
for x in numeric_features:
    fillNA['cat'][x] = '__NA__'

In [ ]:
fillNA

## PIPELINES

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(numeric_features)),
    ('imputer',FillNaDict(col_fill_dict = fillNA['num'])),
    ('binarizer',DFKBinsOrdinalDiscretizer(n_bins=5,cols=numeric_features))
    ])

categorical_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(categorical_features)),
    ('imputer', FillNaDict(col_fill_dict = fillNA['cat'])),
    ('dummies', DummyTransformer(categorical_features))
    ])

sel_ini_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(cols_preproc))
])

# DFFeatureUnion
feat_un = DFFeatureUnion([
            ('categoricals',categorical_transformer),
            ('numeric',numeric_transformer),
            ])

# pipeline_base
pipeline_base = Pipeline(steps=[('seleccionFeatsIni',sel_ini_transformer),
                                ('final_features',feat_un)])
pipeline_base

## Pipeline con MinMaxScaler para vars numéricas

In [ ]:
numeric_transformer_w_scaler = Pipeline(steps=[
    ('extractor',ColumnSelector(numeric_features)),
    ('imputer',FillNaDict(col_fill_dict= fillNA['num'])),
    ('binarizer',MinMaxScalerQ(X_train, columns=numeric_features, q_min = 0.25, q_max = 0.75))
    ])

# DFFeatureUnion
feat_un2 = DFFeatureUnion([
            ('categoricals',categorical_transformer),
            ('numeric',numeric_transformer_w_scaler),
            ])

# pipeline_w_num_scaler
pipeline_w_num_scaler = Pipeline(steps=[('seleccionFeatsIni',sel_ini_transformer),
                                ('final_features',feat_un2)])

In [ ]:
pipeline_w_num_scaler.steps[1][1].transformer_list[1] 

## MODELS

In [ ]:
models = {         
          'xgb':XGBClassifier(),
          
          'lr': LogisticRegression(),
          
          'mlp':MLPClassifier(),
}

In [ ]:
hparams = {'xgb':{'max_depth':[3,5,7,8,10], 'gamma': [1,2,5,10,15,20], 'n_jobs':[7], 'learning_rate': [0.001, 0.01, 0.05, 0.1], 'n_estimators':[100, 150, 200, 300, 500],
                 'colsample_bytree': [0.2, 0.3, 0.5], 'seed': [seed]},
     
          'lr': {'multi_class': ['multinomial', 'ovr'], 'class_weight':['balanced'], 'C': [0.01, 0.1, 1, 10, 20], 'solver': ['saga', 'newton-cg'], 'random_state':[seed]},
    
          
          'mlp':{'solver':["adam", 'sgd'], 'hidden_layer_sizes' : [(64, 64, 64), (64, 64, 64, 64), (64, 64, 64, 64, 64)], 'activation' : ['relu'], 'learning_rate_init' : [0.001, 0.01, 0.05], 'alpha' : [0.0001, 0.05, 0.001],
                 'max_iter':[1800], 'batch_size':[64, 256], 'learning_rate': ['constant','adaptive'], 'random_state':[seed]},         
     
}

In [ ]:
diccionario_entrenamiento = {
                            'xgb': {},
                            'lr': {},
                            'mlp': {},
}
                                                         
for valor in diccionario_entrenamiento.keys():
    diccionario_entrenamiento[valor]['model'] = models[valor]
    diccionario_entrenamiento[valor]['params'] = hparams[valor]


## TRAINING

In [ ]:
X_train = X_train.reset_index(drop=True)

In [ ]:
cv_n = 5 # para aprovechar toda la data ... son muy pocos registros!

In [ ]:
models_path = data_path + "/models/"


In [ ]:
pipeline = pipeline_base

In [ ]:
Y_train_copy = y.copy()

In [ ]:
X_train_copy = X_train.copy()
X_train_copy = pipeline.fit_transform(X_train_copy) 
X_train_copy.reset_index(drop=True, inplace= True)


In [ ]:
path_pipeline = data_path + 'pipeline.joblib'
joblib.dump(pipeline, path_pipeline)


In [ ]:
X_train_copy.head()

In [ ]:
X_train_copy.discount.value_counts()

In [ ]:
X_train_copy['installments.quantity'].describe()

In [ ]:
X_train_copy['installments.rate'].describe()

In [ ]:
X_train_copy['price'].value_counts()

## smote

In [ ]:
oversample = SMOTE(n_jobs=7,
                  sampling_strategy= smote_sampling_strategy)
X_train_copy, Y_train_copy = oversample.fit_resample(X_train_copy, Y_train_copy)


Instancias por clase post resampling:

In [ ]:
Y_train_copy.value_counts().sort_index()

In [ ]:
columns_names = X_train_copy.columns
columns_names

In [ ]:
path_columns = data_path + 'cols.joblib'
joblib.dump(columns_names, path_columns)

In [ ]:
models_path_template = models_path + '{}.joblib'

In [ ]:
basic_opt_params = {'scoring': 'f1_macro', 'cv': cv_n, 'n_jobs': 7}

In [ ]:
for valor in diccionario_entrenamiento.keys():
    model = diccionario_entrenamiento[valor]['model']
    params = diccionario_entrenamiento[valor]['params']
    
    opt = GridSearchCV(estimator=model, param_grid=params, )
    opt.set_params(**basic_opt_params)
    fit = opt.fit(X_train_copy[columns_names], Y_train_copy)
    
    # persisto el best estimator para cada uno
    model_name = models_path_template.format(valor)
    if valor == 'xgb':
        fit.best_estimator_.save_model(model_name)
    else:
        joblib.dump(fit.best_estimator_, model_name)